<a href="https://colab.research.google.com/github/SimonMekonnen/DeepLearing/blob/main/DL_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch

class DenseLayer:
    def __init__(self, input_size, output_size):
        self.weights = 0.01 * torch.randn(output_size, input_size, requires_grad=True)
        self.biases = torch.zeros(output_size, requires_grad=True)

    def forward(self, inputs):
        return torch.mm(inputs, self.weights.t()) + self.biases

In [10]:
class ActivationReLU:
    def forward(self, inputs):
        return torch.max(inputs, torch.tensor(0.0))


In [11]:
class ActivationSoftmax:
    def forward(self, inputs):
        exp_values = torch.exp(inputs - torch.max(inputs, dim=1, keepdim=True)[0])
        probabilities = exp_values / torch.sum(exp_values, dim=1, keepdim=True)
        return probabilities

In [12]:
class ActivationSigmoid:
    def forward(self, inputs):
        return 1 / (1 + torch.exp(-inputs))

In [13]:
class Model:
    def __init__(self, input_size, neurons, output_size, activation_fn):
        self.layers = []
        for neuron in neurons:
            self.layers.append(DenseLayer(input_size, neuron))
            self.layers.append(activation_fn())
            input_size = neuron
        self.layers.append(DenseLayer(neurons[-1], output_size))
        self.layers.append(ActivationSoftmax())

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs


In [14]:
def forward_pass(inputs, model):
    output = inputs
    for layer in model:
        output = layer.forward(output)
    return output


In [15]:
def log_loss(y_true, y_pred):
    y_true_one_hot = torch.zeros_like(y_pred)
    y_true_one_hot.scatter_(1, y_true.unsqueeze(1), 1)
    return -torch.mean(y_true_one_hot * torch.log(y_pred) + (1 - y_true_one_hot) * torch.log(1 - y_pred))


In [16]:
def accuracy(y_true, y_pred):
    pred_index = torch.argmax(y_pred, dim=1)
    correct = pred_index == y_true
    return correct.float().mean()


In [17]:
torch.manual_seed(42)

input_size = 4
neurons = [18, 18,18]
output_size = 3
samples = 10

relu_model = Model(input_size, neurons, output_size, ActivationReLU)
sigmoid_model = Model(input_size, neurons, output_size, ActivationSigmoid)

input_data = torch.randn(samples, input_size)

output_1 = relu_model.forward(input_data)
output_2 = sigmoid_model.forward(input_data)

y_true = torch.randint(0, 3, (samples,)).long()

loss_1 = log_loss(y_true, output_1)
accuracy_1 = accuracy(y_true, output_1)

loss_2 = log_loss(y_true, output_2)
accuracy_2 = accuracy(y_true, output_2)

print(f"ReLU Model - Loss: {loss_1.item()}, Accuracy: {accuracy_1.item()}")
print(f"Sigmoid Model - Loss: {loss_2.item()}, Accuracy: {accuracy_2.item()}")


ReLU Model - Loss: 0.6365141272544861, Accuracy: 0.30000001192092896
Sigmoid Model - Loss: 0.6335100531578064, Accuracy: 0.6000000238418579
